# Final Project Notebook
**Nicholas Peterzell**<br>
**A15278235**

In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from scipy.stats import ttest_ind
from sklearn.metrics import classification_report
from statistics import mean
import itertools

## Importing and cleaning data

In [2]:
# Importing the datasets
bean_df = pd.read_excel('Dry_Bean_Dataset.xlsx')
bike_df = pd.read_csv('bike_hour.csv')
shoppers_df = pd.read_csv('online_shoppers_intention.csv')
stars_df = pd.read_csv('HTRU_2.csv', names=['Mean (integrated profile)',
                                            'Standard deviation (integrated profile)',
                                            'Excess kurtosis (integrated profile)',
                                            'Skewness (integrated profile)',
                                            'Mean (DM-SNR curve)',
                                            'Standard deviation (DM-SNR curve)',
                                            'Excess kurtosis (DM-SNR curve)',
                                            'Skewness (DM-SNR curve)',
                                            'Class'])

In [3]:
# Looking at the datasets
bean_df

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097,759.696,288.721612,185.944705,1.552728,0.765002,42508,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385,DERMASON
13607,42101,757.499,281.576392,190.713136,1.476439,0.735702,42494,231.526798,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219,DERMASON
13608,42139,759.321,281.539928,191.187979,1.472582,0.734065,42569,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767,DERMASON
13609,42147,763.779,283.382636,190.275731,1.489326,0.741055,42667,231.653248,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222,DERMASON


In [4]:
bike_df

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,17375,2012-12-31,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642,11,108,119
17375,17376,2012-12-31,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642,8,81,89
17376,17377,2012-12-31,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,7,83,90
17377,17378,2012-12-31,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,13,48,61


In [5]:
shoppers_df

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.000000,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.000000,0.100000,0.000000,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.000000,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.050000,0.140000,0.000000,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.020000,0.050000,0.000000,0.0,Feb,3,3,1,4,Returning_Visitor,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,3,145.0,0,0.0,53,1783.791667,0.007143,0.029031,12.241717,0.0,Dec,4,6,1,1,Returning_Visitor,True,False
12326,0,0.0,0,0.0,5,465.750000,0.000000,0.021333,0.000000,0.0,Nov,3,2,1,8,Returning_Visitor,True,False
12327,0,0.0,0,0.0,6,184.250000,0.083333,0.086667,0.000000,0.0,Nov,3,2,1,13,Returning_Visitor,True,False
12328,4,75.0,0,0.0,15,346.000000,0.000000,0.021053,0.000000,0.0,Nov,2,2,3,11,Returning_Visitor,False,False


In [6]:
stars_df

,Mean (integrated profile),Standard deviation (integrated profile),Excess kurtosis (integrated profile),Skewness (integrated profile),Mean (DM-SNR curve),Standard deviation (DM-SNR curve),Excess kurtosis (DM-SNR curve),Skewness (DM-SNR curve),Class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0
...,...,...,...,...,...,...,...,...,...
17893,136.429688,59.847421,-0.187846,-0.738123,1.296823,12.166062,15.450260,285.931022,0
17894,122.554688,49.485605,0.127978,0.323061,16.409699,44.626893,2.945244,8.297092,0
17895,119.335938,59.935939,0.159363,-0.743025,21.430602,58.872000,2.499517,4.595173,0
17896,114.507812,53.902400,0.201161,-0.024789,1.946488,13.381731,10.007967,134.238910,0


In [7]:
# bean_df: Changing the labels column (Class) to a binary column with 1 for SEKER class
# and 0 otherwise, and isolating the labels column

bean_df['Class'] = bean_df['Class'].mask(bean_df['Class'] != 'SEKER', 0.0)
bean_df['Class'] = bean_df['Class'].mask(bean_df['Class']
                 == 'SEKER', 1.0).astype(float)
bean_df_y = bean_df['Class']
bean_df = bean_df.drop(['Class'], axis=1)

# bike_df: Isolating the labels column (workingday) in bike_df, 
# and dropping the instant column since it is just a repeat 
# of the index column and the dteday column since it is represented 
# in other columns in numeric form
bike_df_y = bike_df['workingday']
bike_df = bike_df.drop(['workingday'], axis=1)
bike_df = bike_df.drop(['instant', 'dteday'], axis=1)

# shoppers_df: Turning the Month column into numerical data, making the VisitorType,
# Weekend, and Revenue columns 0 and 1 instead of True and False, and isolating
# the labels column
shoppers_df['Month'] = shoppers_df['Month'].replace(['Feb', 'Mar', 'May', 'June',
                                                     'Jul', 'Aug', 'Sep', 'Oct',
                                                     'Nov', 'Dec'], [2, 3, 5, 6, 
                                                      7, 8, 9, 10, 11, 12])
shoppers_df['VisitorType'] = shoppers_df['VisitorType'].mask(shoppers_df['VisitorType']
                           != 'New_Visitor', 0.0)
shoppers_df['VisitorType'] = shoppers_df['VisitorType'].mask(shoppers_df['VisitorType']
                           == 'New_Visitor', 1.0).astype(float)
shoppers_df['Weekend'] = shoppers_df['Weekend'].astype(int)
shoppers_df['Revenue'] = shoppers_df['Revenue'].astype(int)
shoppers_df_y = shoppers_df['VisitorType']
shoppers_df = shoppers_df.drop(['VisitorType'], axis=1)

# stars_df: Isolating the labels column in stars_df
stars_df_y = stars_df['Class']
stars_df = stars_df.drop(['Class'], axis=1)

# All datsets: using StandardScaler() on columns without binary data
bean_scale_cols = ['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength',
                   'AspectRation', 'Eccentricity', 'ConvexArea', 
                   'EquivDiameter', 'Extent', 'Solidity', 'roundness', 'Compactness',
                   'ShapeFactor1', 'ShapeFactor2', 'ShapeFactor3', 'ShapeFactor4']
bean_transformer = ColumnTransformer([('scaler', StandardScaler(), bean_scale_cols)],
                                       remainder='passthrough')
bean_df = bean_transformer.fit_transform(bean_df)

bike_scale_cols = ['season', 'mnth', 'hr', 'weathersit', 'temp', 'atemp',
                   'hum', 'windspeed', 'casual', 'registered', 'cnt']
bike_transformer = ColumnTransformer([('scaler', StandardScaler(), bike_scale_cols)],
                                       remainder='passthrough')
bike_df = bike_transformer.fit_transform(bike_df)

shoppers_scale_cols = ['Administrative', 'Administrative_Duration', 'Informational',
                       'Informational_Duration', 'ProductRelated', 
                       'ProductRelated_Duration', 'BounceRates', 'ExitRates',
                       'SpecialDay', 'Month', 'OperatingSystems', 'Browser', 
                       'Region', 'TrafficType']
shoppers_transformer = ColumnTransformer([('scaler', StandardScaler(),
                                           shoppers_scale_cols)], 
                                           remainder='passthrough')
shoppers_df = shoppers_transformer.fit_transform(shoppers_df)

stars_scale_cols = ['Mean (integrated profile)',
                    'Standard deviation (integrated profile)',
                    'Excess kurtosis (integrated profile)',
                    'Skewness (integrated profile)', 'Mean (DM-SNR curve)',
                    'Standard deviation (DM-SNR curve)',
                    'Excess kurtosis (DM-SNR curve)', 'Skewness (DM-SNR curve)']
stars_transformer = ColumnTransformer([('scaler', StandardScaler(), stars_scale_cols)],
                                      remainder='passthrough')
stars_df = stars_transformer.fit_transform(stars_df)

data = [[bean_df, bean_df_y], [bike_df, bike_df_y],
        [shoppers_df, shoppers_df_y], [stars_df, stars_df_y]]

In [8]:
# Finding % positive on the datasets
# Method of turning decimal to percent is from
# https://www.kite.com/python/answers/how-to-format-a-number-as-a-percentage-in-python
print('Percent positive in the bean dataset is:\n',
      '{:.0%}'.format(bean_df_y.value_counts()[1]
        /(bean_df_y.value_counts()[0]
        + bean_df_y.value_counts()[1])))

print('Percent positive in the bike dataset is:\n',
      '{:.0%}'.format(bike_df_y.value_counts()[1]
        /(bike_df_y.value_counts()[0] + 
        bike_df_y.value_counts()[1])))

print('Percent positive in the shoppers dataset is:\n',
      '{:.0%}'.format(shoppers_df_y.value_counts()[1]
        /(shoppers_df_y.value_counts()[0] + 
        shoppers_df_y.value_counts()[1])))

print('Percent positive in the stars dataset is:\n',
      '{:.0%}'.format(stars_df_y.value_counts()[1]
        /(stars_df_y.value_counts()[0] + 
        stars_df_y.value_counts()[1])))

Percent positive in the bean dataset is:
 15%
Percent positive in the bike dataset is:
 68%
Percent positive in the shoppers dataset is:
 14%
Percent positive in the stars dataset is:
 9%


## Creating the logistic regression classifier

In [9]:
def logreg_clf(data, trials):

    # Initializing the lists that will hold the metric scores
    # for each algorithm's train and test set
    logreg_acc_scores_train = []
    logreg_roc_auc_scores_train = []
    logreg_f1_scores_train = []
    logreg_acc_scores_test = []
    logreg_roc_auc_scores_test = []
    logreg_f1_scores_test = []
    
    for trial in range(trials):
        
        # Making the train-test split for each dataset
        X_train, X_test, y_train, y_test = train_test_split(data[0],
                                                            data[1],
                                                            train_size=5000)
        split = [X_train, X_test, y_train, y_test]
        # Pulling out the proper train/test splits out of the all_splits list
        X_train = split[0]
        X_test = split[1]
        y_train = split[2]
        y_test = split[3]

        # Creating a pipeline with the classifier
        # Pipeline isn't necessary - just reusing code from a previous version
        pipe = Pipeline([('classifier', LogisticRegression())])


        # Creating the C_values for the grid search;
        # newton-cg with a C value of 10^-8 does not converge: this
        # combination is not run to avoid warnings
        C_values = [10**-8, 10**-7, 10**-6, 10**-5, 10**-4, 10**-3, 10**-2,
                    10**-1, 10**0, 10**1, 10**2, 10**3, 10**4]

        # Defining the search space
        search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                         'classifier__solver': ['saga'],
                         'classifier__penalty': ['none', 'l1', 'l2', 'elasticnet'],
                         'classifier__C': C_values},
                        {'classifier': [LogisticRegression(max_iter=10000)],
                         'classifier__solver': ['liblinear'],
                         'classifier__penalty': ['l1', 'l2'],
                         'classifier__C': C_values},
                        {'classifier': [LogisticRegression(max_iter=10000)],
                         'classifier__solver': ['newton-cg', 'lbfgs', 'sag'],
                         'classifier__penalty': ['none', 'l2'],
                         'classifier__C': C_values}]

        # Defining parameters for the grid search
        log_search = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                                  scoring=['accuracy', 'roc_auc', 'f1'], 
                                  refit='accuracy', n_jobs=-1)

        # Fitting the search model to find the best parameters
        log_model = log_search.fit(X_train, y_train)

        # Setting up the classifier with the parameters with the best accuracy
        acc_params = log_model.cv_results_['params'][np.argmin(log_model.cv_results_
                                                            ['rank_test_accuracy'])]
        
        acc_pipe = pipe.set_params(**acc_params)
        acc_pred = pipe.fit(X_train, y_train)
        y_train_pred_acc = acc_pred.predict(X_train)
        y_test_pred_acc = acc_pred.predict(X_test)

        # Setting up the classifier with the parameters with the best roc
        roc_auc_params = log_model.cv_results_['params'][np.argmin(log_model.cv_results_
                                                                ['rank_test_roc_auc'])]
       
        roc_auc_pipe = pipe.set_params(**roc_auc_params)
        roc_auc_pred = roc_auc_pipe.fit(X_train, y_train)
        y_train_pred_roc_auc = roc_auc_pred.predict(X_train)
        y_test_pred_roc_auc = roc_auc_pred.predict(X_test)

        # Setting up the classifier with the parameters with the best f1
        f1_params = log_model.cv_results_['params'][np.argmin(log_model.cv_results_
                                                              ['rank_test_f1'])]
            
        f1_pipe = pipe.set_params(**f1_params)
        f1_pred = f1_pipe.fit(X_train, y_train)
        y_train_pred_f1 = f1_pred.predict(X_train)
        y_test_pred_f1 = f1_pred.predict(X_test)

        # Calculating and appending the logistic regression metrics to their respective
        # lists
        logreg_acc_scores_train.append(accuracy_score(y_train, y_train_pred_acc))
        logreg_roc_auc_scores_train.append(roc_auc_score(y_train, y_train_pred_roc_auc))
        logreg_f1_scores_train.append(f1_score(y_train, y_train_pred_f1))
        logreg_acc_scores_test.append(accuracy_score(y_test, y_test_pred_acc))
        logreg_roc_auc_scores_test.append(roc_auc_score(y_test, y_test_pred_roc_auc))
        logreg_f1_scores_test.append(f1_score(y_test, y_test_pred_f1))

    # Returning a list of the training metrics and testing metrics on the dataset
    return ([logreg_acc_scores_train, logreg_roc_auc_scores_train,
             logreg_f1_scores_train], [logreg_acc_scores_test, 
             logreg_roc_auc_scores_test, logreg_f1_scores_test])

## Creating the decision tree classifier

In [10]:
def DT_clf(data, trials):
    
    # Initializing the lists that will hold the metric scores
    # for each algorithm's train and test set
    DT_acc_scores_train = []
    DT_roc_auc_scores_train = []
    DT_f1_scores_train = []
    DT_acc_scores_test = []
    DT_roc_auc_scores_test = []
    DT_f1_scores_test = []
    
    for trial in range(trials):
        
        # Making the train-test split for each dataset
        X_train, X_test, y_train, y_test = train_test_split(data[0],
                                                            data[1],
                                                            train_size=5000)
        split = [X_train, X_test, y_train, y_test]
    
        # Pulling out the proper train/test splits out of the all_splits list
        X_train = split[0]
        X_test = split[1]
        y_train = split[2]
        y_test = split[3]

        # Creating a pipeline with the classifier
        # Pipeline isn't needed - just reusing code from a previous version
        pipe = Pipeline([('classifier', DecisionTreeClassifier())])

        # Defining the search space
        search_space = [{'classifier': [DecisionTreeClassifier()],
                         'classifier__criterion': ['gini', 'entropy'],
                         'classifier__max_depth': list(range(1,21)),
                         'classifier__ccp_alpha': list(range(1,21)),
                         'classifier__class_weight': [{0:1,1:1},{0:2,1:1},
                                                      {0:1,1:2},{0:5,1:1},
                                                      {0:1,1:5},{0:5,1:2},
                                                      {0:2,1:5},{0:5,1:2},
                                                      {0:10,1:1},{0:1,1:10},
                                                      {0:10,1:2},{0:2,1:10},
                                                      {0:5,1:10},{0:10,1:5}]}]

        # Defining parameters for the grid search
        DT_search = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                                 scoring=['accuracy', 'roc_auc', 'f1'], 
                                 refit='accuracy', n_jobs=-1)

        # Fitting the search model to find the best parameters
        DT_model = DT_search.fit(X_train, y_train)

        # Setting up the classifier with the parameters with the best accuracy score
        acc_params = DT_model.cv_results_['params'][np.argmin(DT_model.cv_results_
                                                            ['rank_test_accuracy'])]
            
        acc_pipe = pipe.set_params(**acc_params)
        acc_pred = pipe.fit(X_train, y_train)
        y_train_pred_acc = acc_pred.predict(X_train)
        y_test_pred_acc = acc_pred.predict(X_test)

        # Setting up the classifier with the parameters with the best roc score
        roc_auc_params = DT_model.cv_results_['params'][np.argmin(DT_model.cv_results_
                                                                ['rank_test_roc_auc'])]
            
        roc_auc_pipe = pipe.set_params(**roc_auc_params)
        roc_auc_pred = roc_auc_pipe.fit(X_train, y_train)
        y_train_pred_roc_auc = roc_auc_pred.predict(X_train)
        y_test_pred_roc_auc = roc_auc_pred.predict(X_test)

        # Setting up the classifier with the parameters with the best f1 score
        f1_params = DT_model.cv_results_['params'][np.argmin(DT_model.cv_results_
                                                            ['rank_test_f1'])]
            
        f1_pipe = pipe.set_params(**f1_params)
        f1_pred = f1_pipe.fit(X_train, y_train)
        y_train_pred_f1 = f1_pred.predict(X_train)
        y_test_pred_f1 = f1_pred.predict(X_test)
        
        # Calculating and appending the DT metrics to their respective lists
        DT_acc_scores_train.append(accuracy_score(y_train, y_train_pred_acc))
        DT_roc_auc_scores_train.append(roc_auc_score(y_train, y_train_pred_roc_auc))
        DT_f1_scores_train.append(f1_score(y_train, y_train_pred_f1))
        DT_acc_scores_test.append(accuracy_score(y_test, y_test_pred_acc))
        DT_roc_auc_scores_test.append(roc_auc_score(y_test, y_test_pred_roc_auc))
        DT_f1_scores_test.append(f1_score(y_test, y_test_pred_f1))
    
    # Returning a list of the training metrics and testing metrics on the dataset
    return ([DT_acc_scores_train, DT_roc_auc_scores_train,
             DT_f1_scores_train], [DT_acc_scores_test, 
             DT_roc_auc_scores_test, DT_f1_scores_test])

## Creating the Random Forest classifier

In [11]:
def RF_clf(data, trials):
    
    # Initializing the lists that will hold the metric scores
    # for each algorithm's train and test set
    RF_acc_scores_train = []
    RF_roc_auc_scores_train = []
    RF_f1_scores_train = []
    RF_acc_scores_test = []
    RF_roc_auc_scores_test = []
    RF_f1_scores_test = []
    
    for trial in range(trials):
        
        # Making the train-test split for each dataset
        X_train, X_test, y_train, y_test = train_test_split(data[0],
                                                            data[1],
                                                            train_size=5000)
        split = [X_train, X_test, y_train, y_test]
    
        # Pulling out the proper train/test splits out of the all_splits list
        X_train = split[0]
        X_test = split[1]
        y_train = split[2]
        y_test = split[3]

        # Creating a pipeline with the classifier
        # Pipeline isn't needed - just reusing code from a previous version
        pipe = Pipeline([('classifier', RandomForestClassifier())])

        # Defining the search space
        search_space = [{'classifier__n_estimators': [1024],
                         'classifier__criterion': ['gini', 'entropy'],
                         'classifier__max_features': [1, 2, 4, 6, 
                                                     8, 12, 16, 20]}]

        # Defining parameters for the grid search
        RF_search = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                                 scoring=['accuracy', 'roc_auc', 'f1'], 
                                 refit='accuracy', n_jobs=-1)

        # Fitting the search model to find the best parameters
        RF_model = RF_search.fit(X_train, y_train)

        # Setting up the classifier with the parameters with the best accuracy score
        acc_params = RF_model.cv_results_['params'][np.argmin(RF_model.cv_results_
                                                            ['rank_test_accuracy'])]
            
        acc_pipe = pipe.set_params(**acc_params)
        acc_pred = pipe.fit(X_train, y_train)
        y_train_pred_acc = acc_pred.predict(X_train)
        y_test_pred_acc = acc_pred.predict(X_test)

        # Setting up the classifier with the parameters with the best roc score
        roc_auc_params = RF_model.cv_results_['params'][np.argmin(RF_model.cv_results_
                                                                ['rank_test_roc_auc'])]
            
        roc_auc_pipe = pipe.set_params(**roc_auc_params)
        roc_auc_pred = roc_auc_pipe.fit(X_train, y_train)
        y_train_pred_roc_auc = roc_auc_pred.predict(X_train)
        y_test_pred_roc_auc = roc_auc_pred.predict(X_test)

        # Setting up the classifier with the parameters with the best f1 score
        f1_params = RF_model.cv_results_['params'][np.argmin(RF_model.cv_results_
                                                            ['rank_test_f1'])]
            
        f1_pipe = pipe.set_params(**f1_params)
        f1_pred = f1_pipe.fit(X_train, y_train)
        y_train_pred_f1 = f1_pred.predict(X_train)
        y_test_pred_f1 = f1_pred.predict(X_test)
        
        # Calculating and appending the DT metrics to their respective lists
        RF_acc_scores_train.append(accuracy_score(y_train, y_train_pred_acc))
        RF_roc_auc_scores_train.append(roc_auc_score(y_train, y_train_pred_roc_auc))
        RF_f1_scores_train.append(f1_score(y_train, y_train_pred_f1))
        RF_acc_scores_test.append(accuracy_score(y_test, y_test_pred_acc))
        RF_roc_auc_scores_test.append(roc_auc_score(y_test, y_test_pred_roc_auc))
        RF_f1_scores_test.append(f1_score(y_test, y_test_pred_f1))
    
    # Returning a list of the training metrics and testing metrics on the dataset
    return ([RF_acc_scores_train, RF_roc_auc_scores_train,
             RF_f1_scores_train], [RF_acc_scores_test, 
             RF_roc_auc_scores_test, RF_f1_scores_test])

## Running the models

In [12]:
# Initializing the lists that will hold the metric scores
# for each algorithm's train and test set
logreg_acc_train = []
logreg_roc_auc_train = []
logreg_f1_train = []
logreg_acc_test = []
logreg_roc_auc_test = []
logreg_f1_test = []

DT_acc_train = []
DT_roc_auc_train = []
DT_f1_train = []
DT_acc_test = []
DT_roc_auc_test = []
DT_f1_test = []

RF_acc_train = []
RF_roc_auc_train = []
RF_f1_train = []
RF_acc_test = []
RF_roc_auc_test = []
RF_f1_test = []

# Setting the number of trials to be run for each algorithm
trials = 5

# Looping over the datasets, running every algorithm on each
for dat in data:
    
    # Initializing lists to hold the metric scores to be pulled out
    # afterwards
    logreg_scores_temp = []
    DT_scores_temp = []
    RF_scores_temp = []
    
    # Appending the logistic regression metrics to their respective lists
    logreg_scores_temp.append(logreg_clf(dat, trials))
    logreg_acc_train.append(logreg_scores_temp[0][0][0])
    logreg_roc_auc_train.append(logreg_scores_temp[0][0][1])
    logreg_f1_train.append(logreg_scores_temp[0][0][2])
    logreg_acc_test.append(logreg_scores_temp[0][1][0])
    logreg_roc_auc_test.append(logreg_scores_temp[0][1][1])
    logreg_f1_test.append(logreg_scores_temp[0][1][2])
    
    # Appending the DT metrics to their respective lists
    DT_scores_temp.append(DT_clf(dat, trials))
    DT_acc_train.append(DT_scores_temp[0][0][0])
    DT_roc_auc_train.append(DT_scores_temp[0][0][1])
    DT_f1_train.append(DT_scores_temp[0][0][2])
    DT_acc_test.append(DT_scores_temp[0][1][0])
    DT_roc_auc_test.append(DT_scores_temp[0][1][1])
    DT_f1_test.append(DT_scores_temp[0][1][2])
    
    # Appending the random forest metrics to their respective lists
    RF_scores_temp.append(RF_clf(dat, trials))
    RF_acc_train.append(RF_scores_temp[0][0][0])
    RF_roc_auc_train.append(RF_scores_temp[0][0][1])
    RF_f1_train.append(RF_scores_temp[0][0][2])
    RF_acc_test.append(RF_scores_temp[0][1][0])
    RF_roc_auc_test.append(RF_scores_temp[0][1][1])
    RF_f1_test.append(RF_scores_temp[0][1][2])

# Making a list to index through and get mean scores of the trials
all_list = [logreg_acc_train, logreg_roc_auc_train,
             logreg_f1_train, logreg_acc_test,
             logreg_roc_auc_test, logreg_f1_test,
             DT_acc_train, DT_roc_auc_train,
             DT_f1_train, DT_acc_test,
             DT_roc_auc_test, DT_f1_test,
             RF_acc_train, RF_roc_auc_train,
             RF_f1_train, RF_acc_test,
             RF_roc_auc_test, RF_f1_test]

C:\Users\npete\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\npete\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\npete\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\npete\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\npete\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\npete\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1320: UserWarning: Setting penalt

In [13]:
# Initializing the list to hold the mean scores across trials
mean_logreg_acc_train = []
mean_logreg_roc_auc_train = []
mean_logreg_f1_train = []
mean_logreg_acc_test = []
mean_logreg_roc_auc_test = []
mean_logreg_f1_test = []

mean_DT_acc_train = []
mean_DT_roc_auc_train = []
mean_DT_f1_train = []
mean_DT_acc_test = []
mean_DT_roc_auc_test = []
mean_DT_f1_test = []

mean_RF_acc_train = []
mean_RF_roc_auc_train = []
mean_RF_f1_train = []
mean_RF_acc_test = []
mean_RF_roc_auc_test = []
mean_RF_f1_test = []

# Idea for average function from
# https://www.geeksforgeeks.org/find-average-list-python/
def list_average(item):
    
    return sum(item)/len(item)

# Calculating the mean of the scores on each dataset
for num in range(4):    
    
    mean_logreg_acc_train.append(list_average(all_list[0][num]))
    mean_logreg_roc_auc_train.append(list_average(all_list[1][num]))
    mean_logreg_f1_train.append(list_average(all_list[2][num]))
    mean_logreg_acc_test.append(list_average(all_list[3][num]))
    mean_logreg_roc_auc_test.append(list_average(all_list[4][num]))
    mean_logreg_f1_test.append(list_average(all_list[5][num]))

    mean_DT_acc_train.append(list_average(all_list[6][num]))
    mean_DT_roc_auc_train.append(list_average(all_list[7][num]))
    mean_DT_f1_train.append(list_average(all_list[8][num]))
    mean_DT_acc_test.append(list_average(all_list[9][num]))
    mean_DT_roc_auc_test.append(list_average(all_list[10][num]))
    mean_DT_f1_test.append(list_average(all_list[11][num]))

    mean_RF_acc_train.append(list_average(all_list[12][num]))
    mean_RF_roc_auc_train.append(list_average(all_list[13][num]))
    mean_RF_f1_train.append(list_average(all_list[14][num]))
    mean_RF_acc_test.append(list_average(all_list[15][num]))
    mean_RF_roc_auc_test.append(list_average(all_list[16][num]))
    mean_RF_f1_test.append(list_average(all_list[17][num]))

## Results dataframes

In [14]:
# Averaging the metric data across datasets and putting them
# into a list to be used in a dataframe
avg_data_logreg_train = [list_average(mean_logreg_acc_train),
                         list_average(mean_logreg_roc_auc_train),
                         list_average(mean_logreg_f1_train)]
avg_data_DT_train = [list_average(mean_DT_acc_train),
                         list_average(mean_DT_roc_auc_train),
                         list_average(mean_DT_f1_train)]
avg_data_RF_train = [list_average(mean_RF_acc_train),
                         list_average(mean_RF_roc_auc_train),
                         list_average(mean_RF_f1_train)]

avg_data_logreg_test = [list_average(mean_logreg_acc_test),
                         list_average(mean_logreg_roc_auc_test),
                         list_average(mean_logreg_f1_test)]
avg_data_DT_test = [list_average(mean_DT_acc_test),
                         list_average(mean_DT_roc_auc_test),
                         list_average(mean_DT_f1_test)]
avg_data_RF_test = [list_average(mean_RF_acc_test),
                         list_average(mean_RF_roc_auc_test),
                         list_average(mean_RF_f1_test)]

# Averaging the dataset results across metrics and putting them
# into a list to be used in a dataframe
# Method used for performing element-wise computations
# https://blog.finxter.com/how-to-average-a-list-of-lists-in-python/
d = [mean_logreg_acc_train, mean_logreg_roc_auc_train, mean_logreg_f1_train]
avg_metric_logreg_train = [sum(a)/len(a) for a in zip(*d)]
d = [mean_DT_acc_train, mean_DT_roc_auc_train, mean_DT_f1_train]
avg_metric_DT_train = [sum(a)/len(a) for a in zip(*d)]
d = [mean_RF_acc_train, mean_RF_roc_auc_train, mean_RF_f1_train]
avg_metric_RF_train = [sum(a)/len(a) for a in zip(*d)]

d = [mean_logreg_acc_test, mean_logreg_roc_auc_test, mean_logreg_f1_test]
avg_metric_logreg_test = [sum(a)/len(a) for a in zip(*d)]
d = [mean_DT_acc_test, mean_DT_roc_auc_test, mean_DT_f1_test]
avg_metric_DT_test = [sum(a)/len(a) for a in zip(*d)]
d = [mean_RF_acc_test, mean_RF_roc_auc_test, mean_RF_f1_test]
avg_metric_RF_test = [sum(a)/len(a) for a in zip(*d)]

print(mean_logreg_roc_auc_train)

[0.9662382054863039, 0.6992271147682532, 0.5170741947842268, 0.8838737142205242]


In [15]:
# Making a dataframe of the training algorithm/metric combinations
df = pd.DataFrame([avg_data_logreg_train, avg_data_DT_train, avg_data_RF_train],
                              index=['acc', 'roc', 'f1'],
                              columns=['Logistic regression', 'Decision trees',
                                       'Random forests'])

df['Logistic regression'] = avg_data_logreg_train
df['Decision trees'] = avg_data_DT_train
df['Random forests'] = avg_data_RF_train
df = df.append(df.mean().rename('mean')).round(3)

df

,Logistic regression,Decision trees,Random forests
acc,0.914,0.826,1.0
roc,0.767,0.500,1.0
f1,0.717,0.362,1.0
mean,0.799,0.563,1.0


In [16]:
# Making a dataframe of the testing algorith/metric combinations
df = pd.DataFrame([avg_data_logreg_test, avg_data_DT_test, avg_data_RF_test],
                              index=['acc', 'roc', 'f1'],
                              columns=['Logistic regression', 'Decision trees',
                                       'Random forests'])

df['Logistic regression'] = avg_data_logreg_test
df['Decision trees'] = avg_data_DT_test
df['Random forests'] = avg_data_RF_test
df = df.append(df.mean().rename('mean')).round(3)

df

,Logistic regression,Decision trees,Random forests
acc,0.914,0.826,0.959
roc,0.768,0.500,0.878
f1,0.713,0.362,0.822
mean,0.798,0.563,0.887


In [17]:
# Making a dataframe of the training algorithm/dataset combinations
df = pd.DataFrame([avg_metric_logreg_train,avg_metric_DT_train,avg_metric_RF_train],
                              index=['Logistic regression', 'Decision trees',
                                     'Random forests'],
                              columns=['bean', 'bike', 'shoppers', 'stars'])
df['mean'] = df.mean(axis=1)
df = df.round(3)

df

,bean,bike,shoppers,stars,mean
Logistic regression,0.967,0.804,0.512,0.913,0.799
Decision trees,0.537,0.665,0.534,0.515,0.563
Random forests,1.000,1.000,1.000,1.000,1.000


In [18]:
# Making a dataframe of the testing algorithm/dataset combinations
df = pd.DataFrame([avg_metric_logreg_test,avg_metric_DT_test,avg_metric_RF_test],
                              index=['Logistic regression', 'Decision trees',
                                     'Random forests'],
                              columns=['bean', 'bike', 'shoppers', 'stars'])
df['mean'] = df.mean(axis=1)
df = df.round(3)

df

,bean,bike,shoppers,stars,mean
Logistic regression,0.968,0.805,0.504,0.916,0.798
Decision trees,0.537,0.665,0.535,0.514,0.563
Random forests,0.964,1.000,0.657,0.925,0.887


In [50]:
# Finding the p-values between algorithm metrics
# Method of combining lists inide a list from
# https://stackoverflow.com/questions/952914/
# how-to-make-a-flat-list-out-of-list-of-lists?rq=1

print('\nTable 2 p-values:')

print('\nTesting p-value between logistic regression and random forest on the bean data:\n',
     ttest_ind(logreg_acc_test[0] + logreg_roc_auc_test[0] + logreg_f1_test[0],
              RF_acc_test[0] + RF_roc_auc_test[0] + RF_f1_test[0])[1])

print('\nTesting p-value between logistic regression and decision tree on the bean data:\n',
     ttest_ind(logreg_acc_test[0] + logreg_roc_auc_test[0] + logreg_f1_test[0],
              DT_acc_test[0] + DT_roc_auc_test[0] + DT_f1_test[0])[1])

print('\nTesting p-value between random forest and logistic regression on the bike data:\n',
     ttest_ind(RF_acc_test[1] + RF_roc_auc_test[1] + RF_f1_test[1],
              logreg_acc_test[1] + logreg_roc_auc_test[1] + logreg_f1_test[1])[1])

print('\nTesting p-value between random forest and decision tree on the bike data:\n',
     ttest_ind(RF_acc_test[1] + RF_roc_auc_test[1] + RF_f1_test[1],
              DT_acc_test[1] + DT_roc_auc_test[1] + DT_f1_test[1])[1])

print('\nTesting p-value between random forest and logistic regression on the shoppers data:\n',
     ttest_ind(RF_acc_test[2] + RF_roc_auc_test[2] + RF_f1_test[2],
              logreg_acc_test[2] + logreg_roc_auc_test[2] + logreg_f1_test[2])[1])

print('\nTesting p-value between random forest and decision tree on the shoppers data:\n',
     ttest_ind(RF_acc_test[2] + RF_roc_auc_test[2] + RF_f1_test[2],
              DT_acc_test[2] + DT_roc_auc_test[2] + DT_f1_test[2])[1])

print('\nTesting p-value between random forest and logistic regression on the stars data:\n',
     ttest_ind(RF_acc_test[3] + RF_roc_auc_test[3] + RF_f1_test[3],
              logreg_acc_test[3] + logreg_roc_auc_test[3] + logreg_f1_test[3])[1])

print('\nTesting p-value between random forest and decision tree on the stars data:\n',
     ttest_ind(RF_acc_test[3] + RF_roc_auc_test[3] + RF_f1_test[3],
              DT_acc_test[3] + DT_roc_auc_test[3] + DT_f1_test[3])[1])

print('\nTesting p-value between the mean of random forest and the mean of logistic regression:\n',
      ttest_ind(RF_acc_test[1] + RF_roc_auc_test[1] + RF_f1_test[1]
               + RF_acc_test[2] + RF_roc_auc_test[2] + RF_f1_test[2]
               + RF_acc_test[3] + RF_roc_auc_test[3] + RF_f1_test[3],
               logreg_acc_test[1] + logreg_roc_auc_test[1] + logreg_f1_test[1]
               + logreg_acc_test[2] + logreg_roc_auc_test[2] + logreg_f1_test[2]
               + logreg_acc_test[3] + logreg_roc_auc_test[3] + logreg_f1_test[3])[1])

print('\nTesting p-value between the mean of random forest and the mean of decision tree:\n',
      ttest_ind(RF_acc_test[1] + RF_roc_auc_test[1] + RF_f1_test[1]
               + RF_acc_test[2] + RF_roc_auc_test[2] + RF_f1_test[2]
               + RF_acc_test[3] + RF_roc_auc_test[3] + RF_f1_test[3],
               DT_acc_test[1] + DT_roc_auc_test[1] + DT_f1_test[1]
               + DT_acc_test[2] + DT_roc_auc_test[2] + DT_f1_test[2]
               + DT_acc_test[3] + DT_roc_auc_test[3] + DT_f1_test[3])[1])

print('\nTable 3 p-values:\n')

print('p-value between random forest accuracy and logistic regression accuracy:\n',
      ttest_ind(list(itertools.chain(*RF_acc_test)),
          list(itertools.chain(*logreg_acc_test)))[1])

print('\np-value between random forest accuracy and decision tree accuracy:\n',
      ttest_ind(list(itertools.chain(*RF_acc_test)),
          list(itertools.chain(*DT_acc_test)))[1])

print('\np-value between random forest ROC and logistic regression ROC:\n',
      ttest_ind(list(itertools.chain(*RF_roc_auc_test)),
          list(itertools.chain(*logreg_roc_auc_test)))[1])

print('\np-value between random forest ROC and decision tree ROC:\n',
      ttest_ind(list(itertools.chain(*RF_roc_auc_test)),
          list(itertools.chain(*DT_roc_auc_test)))[1])

print('\np-value between random forest f1 and logistic regression f1:\n',
      ttest_ind(list(itertools.chain(*RF_f1_test)),
          list(itertools.chain(*logreg_f1_test)))[1])

print('\np-value between random forest f1 and decision tree f1:\n',
      ttest_ind(list(itertools.chain(*RF_f1_test)),
          list(itertools.chain(*DT_f1_test)))[1])

print('\np-value between random forest mean and logistic regression mean:\n',
      ttest_ind(list(itertools.chain(*RF_acc_test))
     + list(itertools.chain(*RF_roc_auc_test))
     + list(itertools.chain(*RF_f1_test)), list(itertools.chain(*logreg_acc_test))
     + list(itertools.chain(*logreg_roc_auc_test))
     + list(itertools.chain(*logreg_f1_test)))[1])

print('\np-value between random forest mean and decision tree mean:\n',
      ttest_ind(list(itertools.chain(*RF_acc_test))
      + list(itertools.chain(*RF_roc_auc_test))
     + list(itertools.chain(*RF_f1_test)), list(itertools.chain(*DT_acc_test))
     + list(itertools.chain(*DT_roc_auc_test))
     + list(itertools.chain(*DT_f1_test)))[1])

print('\nTable 4 p-values:')

print('\nTraining p-value between random forest and logistic regression on the bean data:\n',
     ttest_ind(RF_acc_train[0] + RF_roc_auc_train[0] + RF_f1_train[0],
              logreg_acc_train[0] + logreg_roc_auc_train[0] + logreg_f1_train[0])[1])

print('\nTraining p-value between logistic regression and decision tree on the bean data:\n',
     ttest_ind(logreg_acc_test[0] + logreg_roc_auc_test[0] + logreg_f1_test[0],
              DT_acc_train[0] + DT_roc_auc_train[0] + DT_f1_train[0])[1])

print('\nTraining p-value between random forest and logistic regression on the bike data:\n',
     ttest_ind(RF_acc_train[1] + RF_roc_auc_train[1] + RF_f1_train[1],
              logreg_acc_train[1] + logreg_roc_auc_train[1] + logreg_f1_train[1])[1])

print('\nTraining p-value between random forest and decision tree on the bike data:\n',
     ttest_ind(RF_acc_train[1] + RF_roc_auc_train[1] + RF_f1_train[1],
              DT_acc_train[1] + DT_roc_auc_train[1] + DT_f1_train[1])[1])

print('\nTraining p-value between random forest and logistic regression on the shoppers data:\n',
     ttest_ind(RF_acc_train[2] + RF_roc_auc_train[2] + RF_f1_train[2],
              logreg_acc_train[2] + logreg_roc_auc_train[2] + logreg_f1_train[2])[1])

print('\nTraining p-value between random forest and decision tree on the shoppers data:\n',
     ttest_ind(RF_acc_train[2] + RF_roc_auc_train[2] + RF_f1_train[2],
              DT_acc_train[2] + DT_roc_auc_train[2] + DT_f1_train[2])[1])

print('\nTraining p-value between random forest and logistic regression on the stars data:\n',
     ttest_ind(RF_acc_train[3] + RF_roc_auc_train[3] + RF_f1_train[3],
              logreg_acc_train[3] + logreg_roc_auc_train[3] + logreg_f1_train[3])[1])

print('\nTraining p-value between random forest and decision tree on the stars data:\n',
     ttest_ind(RF_acc_train[3] + RF_roc_auc_train[3] + RF_f1_train[3],
              DT_acc_train[3] + DT_roc_auc_train[3] + DT_f1_train[3])[1])

print('\nTraining p-value between the mean of random forest and the mean of logistic regression:\n',
      ttest_ind(RF_acc_train[1] + RF_roc_auc_train[1] + RF_f1_train[1]
               + RF_acc_train[2] + RF_roc_auc_train[2] + RF_f1_train[2]
               + RF_acc_train[3] + RF_roc_auc_train[3] + RF_f1_train[3],
               logreg_acc_train[1] + logreg_roc_auc_train[1] + logreg_f1_train[1]
               + logreg_acc_train[2] + logreg_roc_auc_train[2] + logreg_f1_train[2]
               + logreg_acc_train[3] + logreg_roc_auc_train[3] + logreg_f1_train[3])[1])

print('\nTraining p-value between the mean of random forest and the mean of decision tree:\n',
      ttest_ind(RF_acc_train[1] + RF_roc_auc_train[1] + RF_f1_train[1]
               + RF_acc_train[2] + RF_roc_auc_train[2] + RF_f1_train[2]
               + RF_acc_train[3] + RF_roc_auc_train[3] + RF_f1_train[3],
               DT_acc_train[1] + DT_roc_auc_train[1] + DT_f1_train[1]
               + DT_acc_train[2] + DT_roc_auc_train[2] + DT_f1_train[2]
               + DT_acc_train[3] + DT_roc_auc_train[3] + DT_f1_train[3])[1])


Table 2 p-values:

Testing p-value between logistic regression and random forest on the bean data:
 0.45538329188490956

Testing p-value between logistic regression and decision tree on the bean data:
 3.510175484062577e-07

Testing p-value between random forest and logistic regression on the bike data:
 4.865508201408333e-10

Testing p-value between random forest and decision tree on the bike data:
 1.4735748423265126e-10

Testing p-value between random forest and logistic regression on the shoppers data:
 0.10618497386580875

Testing p-value between random forest and decision tree on the shoppers data:
 0.14423704773003818

Testing p-value between random forest and logistic regression on the stars data:
 0.6071199118028152

Testing p-value between random forest and decision tree on the stars data:
 5.023997542860683e-05

Testing p-value between the mean of random forest and the mean of logistic regression:
 0.011925362902675997

Testing p-value between the mean of random forest and 